# Phase 2v2: RPM 기반 고도화 Instruction 데이터셋 구축

## 📌 목적
RPM(Reasoning-Level Personalization, arXiv:2505.21082) 논문의 방법론을 적용하여,
인기글의 **인기 요인을 피처로 추출 → 팩터로 클러스터링 → 추론경로(CoT) 생성**하고,
SOS/EOS 토큰이 포함된 고품질 학습 데이터셋을 구축합니다.

### RPM 파이프라인
```
Step 1: Feature Extraction    — 각 인기글에서 인기 기여 피처를 Gemini로 추출
Step 2: Factor Construction   — 피처들을 LLM 기반 클러스터링(PAS)으로 팩터 구성
Step 3: Reasoning Path        — 팩터 기반으로 추론경로 생성 (= CoT)
Step 4: Dataset Generation    — SOS/EOS + CoT 통합 JSONL 생성
```

## ⚙️ 사전 조건
- `phase0_preprocessing_v2.ipynb` 실행으로 `outputs/인기글_clean_v2.parquet` 존재
- Colab 시크릿에 `GEMINI_API_KEY` 등록 (좌측 🔑 아이콘)

## 📂 출력 파일
```
outputs/
├── rpm_features.json            # Step 1: 추출된 피처
├── rpm_factors.json             # Step 2: 클러스터링된 팩터
├── rpm_reasoning_paths.json     # Step 3: 추론경로
└── training_dataset_v2.jsonl    # Step 4: 최종 학습 데이터셋
```

---
## 0. 패키지 설치 및 환경 설정

In [ ]:
%pip install -q pandas numpy google-generativeai tqdm

print("✅ 패키지 설치 완료")

In [ ]:
from google.colab import drive, userdata

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

In [ ]:
import google.generativeai as genai
import json
import time
import re
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from IPython.display import display

# ── Gemini API 설정 ───────────────────────────────────────
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel('gemini-flash-latest')

print("✅ Gemini API 연결 완료")

In [ ]:
for m in genai.list_models():
    if "generateContent" in [ms for ms in m.supported_generation_methods]:
        print(m.name)

In [ ]:
# ── 경로 설정 ────────────────────────────────────────────
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 데이터 로드 ──────────────────────────────────────────
HOT_PARQUET = OUTPUT_DIR / "인기글_clean_v2.parquet"
if not HOT_PARQUET.exists():
    raise FileNotFoundError(
        f"{HOT_PARQUET}를 찾을 수 없습니다. phase0_preprocessing_v2.ipynb를 먼저 실행하세요."
    )

hot_df = pd.read_parquet(HOT_PARQUET)
print(f"✅ 인기글 데이터 로드 완료: {len(hot_df):,}행")

# engagement score 기반 상위 2,000개 필터링
# 추천수 + 댓글수×0.5 가중합으로 "인기도"가 높은 글 우선 선택
TOP_K = 2000
# Fix: Ensure a Series of 0s is used if the column doesn't exist
recommend_values = hot_df.get("recommend", pd.Series(0, index=hot_df.index))
hot_df["recommend"] = pd.to_numeric(recommend_values, errors="coerce").fillna(0)

comment_count_values = hot_df.get("comment_count", pd.Series(0, index=hot_df.index))
hot_df["comment_count"] = pd.to_numeric(comment_count_values, errors="coerce").fillna(0)

hot_df["engagement_score"] = hot_df["recommend"] + hot_df["comment_count"] * 0.5
# 게시판별 비례 샘플링 (특정 게시판 쏠림 방지)
hot_df = (
    hot_df
    .sort_values("engagement_score", ascending=False)
    .groupby("board", group_keys=False)
    .apply(lambda g: g.head(max(1, int(len(g) / len(hot_df) * TOP_K))))
)
print(f"✅ 상위 {len(hot_df):,}건 필터링 완료 (engagement_score 기준)")
# ── 🔼 여기까지 추가 ──────────────────────────────────────
print(f"📊 게시판 분포:")
print(hot_df["board"].value_counts())

---
## 1. Gemini API 헬퍼 함수

In [ ]:
def call_gemini(prompt: str, max_retries: int = 3, temperature: float = 0.0) -> str:
    """
    Gemini API 호출 (재시도 + Rate Limit 대응).
    """
    for attempt in range(max_retries):
        try:
            response = gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=temperature,
                    max_output_tokens=1024,
                ),
            )
            return response.text.strip()
        except Exception as e:
            error_str = str(e).lower()
            if "rate" in error_str or "quota" in error_str or "429" in error_str:
                wait_time = 15 * (attempt + 1)
                print(f"  ⏳ Rate limit, {wait_time}초 대기 (시도 {attempt+1}/{max_retries})")
                time.sleep(wait_time)
            else:
                print(f"  ⚠️ Gemini 호출 실패: {e}")
                if attempt < max_retries - 1:
                    time.sleep(5)
                else:
                    return ""
    return ""


def parse_json_response(text: str) -> dict | list | None:
    """
    Gemini 응답에서 JSON을 추출.
    ```json ... ``` 블록이 있으면 그 안의 내용을, 없으면 전체를 파싱 시도.
    """
    # markdown json 블록 추출
    match = re.search(r'```(?:json)?\s*\n?(.*?)\n?```', text, re.DOTALL)
    if match:
        text = match.group(1).strip()

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # JSON 부분만 추출 시도
        for start_char, end_char in [('{', '}'), ('[', ']')]:
            start = text.find(start_char)
            end = text.rfind(end_char)
            if start != -1 and end != -1 and end > start:
                try:
                    return json.loads(text[start:end+1])
                except json.JSONDecodeError:
                    continue
        return None


print("✅ 헬퍼 함수 정의 완료")

---
## 2. RPM Step 1: Feature Extraction (피처 추출)

각 인기글에서 **"인기에 기여하는 요소(response-influential features)"**를 Gemini로 추출합니다.

RPM 논문의 `G_qi = M(qi)` 에 해당하며, 각 피처는 `(name, context)` 쌍으로 구성됩니다.

In [ ]:
FEATURE_EXTRACTION_PROMPT = """
당신은 대학 커뮤니티 '서담'의 인기글 분석 전문가입니다.

다음 인기글에서, 이 글이 인기를 얻는 데 기여한 핵심 요소(feature)들을 추출하세요.
각 피처는 글의 내용/구조/스타일에서 독자의 관심과 공감을 이끌어내는 요인입니다.

[게시판] {board}
[제목] {title}
[내용]
{contents}

3~5개의 피처를 추출하여 다음 JSON 형식으로만 응답하세요:
{{
  "features": [
    {{"feature_name": "피처의 의미적 라벨", "context": "이 피처가 글에서 어떻게 나타나는지 구체적 설명"}},
    ...
  ]
}}
"""

print("✅ Feature Extraction 프롬프트 정의 완료")

In [ ]:
# ── 피처 추출 실행 ────────────────────────────────────────
# 캐시 파일이 있으면 로드, 없으면 새로 추출
FEATURES_CACHE = OUTPUT_DIR / "rpm_features.json"

if FEATURES_CACHE.exists():
    with open(FEATURES_CACHE, "r", encoding="utf-8") as f:
        all_features = json.load(f)
    print(f"✅ 캐시에서 피처 로드: {len(all_features)}건")
else:
    all_features = {}  # key: post_key(str), value: {"features": [...]}

# 아직 추출 안 된 게시글만 처리
remaining = hot_df[~hot_df["post_key"].astype(str).isin(all_features.keys())]
print(f"📝 추출 대상: {len(remaining):,}건 (이미 완료: {len(all_features):,}건)")

# 처리 + 주기적 저장
SAVE_INTERVAL = 50
for idx, (_, row) in enumerate(tqdm(remaining.iterrows(), total=len(remaining), desc="피처 추출")):
    post_key = str(row["post_key"])
    title = str(row.get("title", "")).strip()
    contents = str(row.get("contents", "")).strip()[:1500]  # 토큰 제한
    board = str(row.get("board", "")).strip()

    prompt = FEATURE_EXTRACTION_PROMPT.format(
        board=board, title=title, contents=contents
    )

    response_text = call_gemini(prompt)
    parsed = parse_json_response(response_text)

    if parsed and "features" in parsed:
        all_features[post_key] = parsed
    else:
        # 파싱 실패 시 기본 피처
        all_features[post_key] = {
            "features": [
                {"feature_name": "내용_구성", "context": title}
            ]
        }

    # 주기적 저장 (중단 대비)
    if (idx + 1) % SAVE_INTERVAL == 0:
        with open(FEATURES_CACHE, "w", encoding="utf-8") as f:
            json.dump(all_features, f, ensure_ascii=False, indent=2)
        print(f"  💾 중간 저장 ({len(all_features)}건)")

    # Rate limit 방지
    time.sleep(0.3)

# 최종 저장
with open(FEATURES_CACHE, "w", encoding="utf-8") as f:
    json.dump(all_features, f, ensure_ascii=False, indent=2)

print(f"\n✅ 피처 추출 완료: {len(all_features)}건 → {FEATURES_CACHE}")

In [ ]:
# ── 피처 추출 결과 샘플 확인 ──────────────────────────────
sample_keys = list(all_features.keys())[:3]
for key in sample_keys:
    row = hot_df[hot_df["post_key"].astype(str) == key].iloc[0]
    print(f"\n📝 [{row['board']}] {row['title'][:50]}")
    for feat in all_features[key]["features"]:
        print(f"  → {feat['feature_name']}: {feat['context'][:80]}")

---
## 3. RPM Step 2: Factor Construction (팩터 구성)

추출된 피처들을 **LLM 기반 시맨틱 클러스터링(PAS: Propose-Assign-Select)**으로
상위 팩터(Factor)로 그룹화합니다.

RPM 논문 Section 3.2의 Factor Generation 프로세스에 해당합니다.

In [ ]:
# ── 전체 피처 풀 구성 ─────────────────────────────────────
feature_pool = []  # [{"feature_name": ..., "context": ..., "post_key": ...}, ...]
for post_key, data in all_features.items():
    for feat in data.get("features", []):
        feature_pool.append({
            "feature_name": feat.get("feature_name", ""),
            "context": feat.get("context", ""),
            "post_key": post_key,
        })

print(f"📊 전체 피처 풀: {len(feature_pool):,}개")

# 피처 이름 빈도 확인
name_counts = Counter(f["feature_name"] for f in feature_pool)
print("\n🔍 상위 20개 피처 이름:")
for name, count in name_counts.most_common(20):
    print(f"  {name}: {count}회")

In [ ]:
# ── PAS Step 1: Propose (팩터 후보 제안) ───────────────────
FACTORS_CACHE = OUTPUT_DIR / "rpm_factors.json"

if FACTORS_CACHE.exists():
    with open(FACTORS_CACHE, "r", encoding="utf-8") as f:
        factors_data = json.load(f)
    print(f"✅ 캐시에서 팩터 로드: {len(factors_data['factors'])}개 팩터")
else:
    # 랜덤 샘플 30%로 팩터 후보 제안
    np.random.seed(42)
    sample_size = min(len(feature_pool), max(100, int(len(feature_pool) * 0.3)))
    sampled_features = np.random.choice(
        [f"{f['feature_name']}: {f['context'][:60]}" for f in feature_pool],
        size=sample_size,
        replace=False
    ).tolist()

    propose_prompt = f"""
당신은 대학 커뮤니티 인기글 분석 전문가입니다.

다음은 서담 커뮤니티 인기글에서 추출된 피처(인기 기여 요소)들의 샘플입니다:

{chr(10).join(f'- {feat}' for feat in sampled_features[:80])}

이 피처들을 의미적으로 유사한 그룹으로 묶을 수 있는 상위 팩터(Factor)를 5~8개 제안하세요.
각 팩터는 인기글의 인기 요인 유형을 대표해야 합니다.

다음 JSON 형식으로만 응답하세요:
{{
  "factors": [
    {{
      "factor_name": "팩터 이름",
      "description": "이 팩터가 의미하는 바에 대한 설명"
    }},
    ...
  ]
}}
"""

    propose_response = call_gemini(propose_prompt, temperature=0.5)
    proposed_factors = parse_json_response(propose_response)

    if proposed_factors and "factors" in proposed_factors:
        print(f"✅ 팩터 후보 제안 완료: {len(proposed_factors['factors'])}개")
        for f in proposed_factors["factors"]:
            print(f"  → {f['factor_name']}: {f['description'][:60]}")
    else:
        # 제안 실패 시 기본 팩터
        proposed_factors = {
            "factors": [
                {"factor_name": "감정적_공감", "description": "독자의 감정적 공감을 유발하는 내용이나 어투"},
                {"factor_name": "유머_반전", "description": "웃음을 유발하는 유머러스한 내용이나 반전 요소"},
                {"factor_name": "정보_전달", "description": "유용한 정보를 효과적으로 전달하는 구조나 내용"},
                {"factor_name": "참여_유도", "description": "질문/투표 등으로 독자의 적극적 참여를 유도"},
                {"factor_name": "일상_공유", "description": "대학생 일상의 보편적 경험을 공유하여 공감 획득"},
                {"factor_name": "논란_이슈", "description": "의견이 갈리는 주제로 토론을 유발"},
            ]
        }
        print("⚠️ 제안 실패, 기본 팩터 사용")

    factors_data = proposed_factors  # 초기화

In [ ]:
# ── PAS Step 2: Assign (피처 → 팩터 할당) ─────────────────
# 대량의 피처를 배치로 처리
factor_names = [f["factor_name"] for f in factors_data["factors"]]
factor_descriptions = {
    f["factor_name"]: f.get("description", "")
    for f in factors_data["factors"]
}

# 이미 할당 결과가 있으면 스킵
if "assignments" not in factors_data:
    factors_data["assignments"] = {}  # post_key -> [{feature_name, factor}]

BATCH_SIZE = 20
unassigned_posts = [
    pk for pk in all_features.keys()
    if pk not in factors_data.get("assignments", {})
]
print(f"📝 할당 대상: {len(unassigned_posts):,}건")

factor_list_str = "\n".join(
    f"{i+1}. {name}: {factor_descriptions[name][:60]}"
    for i, name in enumerate(factor_names)
)

for batch_start in tqdm(range(0, len(unassigned_posts), BATCH_SIZE), desc="팩터 할당"):
    batch_keys = unassigned_posts[batch_start:batch_start + BATCH_SIZE]

    for post_key in batch_keys:
        features = all_features[post_key].get("features", [])
        if not features:
            factors_data["assignments"][post_key] = []
            continue

        features_str = "\n".join(
            f"  {i+1}. {f['feature_name']}: {f.get('context', '')[:60]}"
            for i, f in enumerate(features)
        )

        assign_prompt = f"""
다음 피처들을 가장 적합한 팩터에 할당하세요.

[사용 가능한 팩터]
{factor_list_str}

[할당할 피처들]
{features_str}

각 피처에 대해 가장 적합한 팩터 이름을 할당하여 다음 JSON으로 응답하세요:
{{
  "assignments": [
    {{"feature_name": "...", "factor": "할당된 팩터 이름"}},
    ...
  ]
}}
"""
        resp = call_gemini(assign_prompt)
        parsed = parse_json_response(resp)

        if parsed and "assignments" in parsed:
            factors_data["assignments"][post_key] = parsed["assignments"]
        else:
            # 실패 시 첫 번째 팩터에 할당
            factors_data["assignments"][post_key] = [
                {"feature_name": f["feature_name"], "factor": factor_names[0]}
                for f in features
            ]

        time.sleep(0.3)

    # 배치 단위 중간 저장
    with open(FACTORS_CACHE, "w", encoding="utf-8") as f:
        json.dump(factors_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ 팩터 할당 완료 → {FACTORS_CACHE}")

In [ ]:
# ── 팩터 통계 계산 ────────────────────────────────────────
# RPM 논문: coverage, influence_rate 등 통계 부여
factor_stats = {name: {"count": 0, "posts": set()} for name in factor_names}

for post_key, assignments in factors_data.get("assignments", {}).items():
    for a in assignments:
        factor = a.get("factor", "")
        if factor in factor_stats:
            factor_stats[factor]["count"] += 1
            factor_stats[factor]["posts"].add(post_key)

total_posts = len(all_features)
print("\n📊 팩터 통계:")
print(f"{'팩터':<20} {'피처수':>8} {'커버리지':>10} {'게시글수':>10}")
print("-" * 50)

for name in factor_names:
    stats = factor_stats[name]
    coverage = len(stats["posts"]) / max(total_posts, 1)
    factor_stats[name]["coverage"] = round(coverage, 3)
    factor_stats[name]["post_count"] = len(stats["posts"])
    # set은 JSON 저장 불가하므로 제거
    factor_stats[name]["posts"] = list(stats["posts"])[:10]  # 대표 게시글만
    print(f"{name:<20} {stats['count']:>8} {coverage:>10.1%} {stats['post_count']:>10}")

# 팩터 통계를 factors_data에 추가
factors_data["factor_stats"] = {
    name: {k: v for k, v in stats.items() if k != "posts"}
    for name, stats in factor_stats.items()
}

with open(FACTORS_CACHE, "w", encoding="utf-8") as f:
    json.dump(factors_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ 팩터 통계 저장 완료")

---
## 4. RPM Step 3: Personalized Reasoning Path (추론경로 생성 = CoT)

각 인기글에 대해 피처와 팩터 정보를 기반으로
**"왜 이 글이 인기인지"를 단계적으로 추론하는 경로**를 생성합니다.

RPM 논문 Section 3.3의 `r_qi = M(qi, G_qi, C_u, ai)`에 해당합니다.

In [ ]:
REASONING_PROMPT = """
당신은 대학 커뮤니티 인기글 분석 전문가입니다.

다음 인기글의 피처와 팩터 정보를 바탕으로,
이 글이 인기를 얻게 된 이유를 단계적으로 추론하세요.

[게시판] {board}
[제목] {title}
[내용 요약] {contents_summary}

[추출된 피처]
{features_str}

[해당 팩터]
{factor_str}

위 정보를 종합하여, 이 글을 작성할 때 어떤 전략을 사용하면
인기글이 될 수 있는지 2~3문장으로 추론하세요.
"왜 이런 요소가 효과적인지" + "글을 쓸 때의 전략적 접근"을 포함하세요.

다음 JSON 형식으로만 응답하세요:
{{"reasoning": "추론 내용"}}
"""

print("✅ Reasoning Path 프롬프트 정의 완료")

In [ ]:
# ── 추론경로 생성 실행 ────────────────────────────────────
REASONING_CACHE = OUTPUT_DIR / "rpm_reasoning_paths.json"

if REASONING_CACHE.exists():
    with open(REASONING_CACHE, "r", encoding="utf-8") as f:
        reasoning_paths = json.load(f)
    print(f"✅ 캐시에서 추론경로 로드: {len(reasoning_paths)}건")
else:
    reasoning_paths = {}

remaining_keys = [
    pk for pk in all_features.keys()
    if pk not in reasoning_paths
]
print(f"📝 추론경로 생성 대상: {len(remaining_keys):,}건")

for idx, post_key in enumerate(tqdm(remaining_keys, desc="추론경로 생성")):
    row_match = hot_df[hot_df["post_key"].astype(str) == post_key]
    if row_match.empty:
        continue
    row = row_match.iloc[0]

    title = str(row.get("title", "")).strip()
    contents = str(row.get("contents", "")).strip()
    contents_summary = contents[:300]  # 요약
    board = str(row.get("board", "")).strip()

    # 피처 문자열
    features = all_features.get(post_key, {}).get("features", [])
    features_str = "\n".join(
        f"- {f['feature_name']}: {f.get('context', '')[:60]}"
        for f in features
    )

    # 팩터 정보
    assignments = factors_data.get("assignments", {}).get(post_key, [])
    assigned_factors = list(set(a.get("factor", "") for a in assignments if a.get("factor")))
    factor_str = ", ".join(assigned_factors) if assigned_factors else "일반"

    prompt = REASONING_PROMPT.format(
        board=board, title=title, contents_summary=contents_summary,
        features_str=features_str, factor_str=factor_str
    )

    resp = call_gemini(prompt)
    parsed = parse_json_response(resp)

    if parsed and "reasoning" in parsed:
        reasoning_paths[post_key] = {
            "reasoning": parsed["reasoning"],
            "factors": assigned_factors,
        }
    else:
        # 파싱 실패 시 응답 텍스트를 그대로 사용
        reasoning_paths[post_key] = {
            "reasoning": resp[:300] if resp else f"{factor_str} 팩터에 해당하는 인기글입니다.",
            "factors": assigned_factors,
        }

    # 주기적 저장
    if (idx + 1) % SAVE_INTERVAL == 0:
        with open(REASONING_CACHE, "w", encoding="utf-8") as f:
            json.dump(reasoning_paths, f, ensure_ascii=False, indent=2)
        print(f"  💾 중간 저장 ({len(reasoning_paths)}건)")

    time.sleep(0.3)

# 최종 저장
with open(REASONING_CACHE, "w", encoding="utf-8") as f:
    json.dump(reasoning_paths, f, ensure_ascii=False, indent=2)

print(f"\n✅ 추론경로 생성 완료: {len(reasoning_paths)}건 → {REASONING_CACHE}")

In [ ]:
# ── 추론경로 샘플 확인 ────────────────────────────────────
sample_keys = list(reasoning_paths.keys())[:3]
for key in sample_keys:
    row = hot_df[hot_df["post_key"].astype(str) == key]
    if row.empty:
        continue
    row = row.iloc[0]
    rp = reasoning_paths[key]
    print(f"\n📝 [{row['board']}] {row['title'][:50]}")
    print(f"  팩터: {', '.join(rp['factors'])}")
    print(f"  추론: {rp['reasoning'][:150]}...")

---
## 5. 키워드 추출 (기존 Phase 2 호환)

In [ ]:
from collections import Counter

DEFAULT_STOPWORDS = [
    "이","가","을","를","에","의","와","과","도","로","으로",
    "은","는","에서","에게","께","한테",
    "더","그","저","이것","그것","저것","그런","이런","저런",
    "때","때문","것","수","등",
    "있다","없다","하다","되다","이다","아니다","같다",
    "게시판","게시글","글","댓글","작성","작성자","조회","추천","비추천",
    "nan", "NaN", "None",
]
CUSTOM_STOPWORDS = set(DEFAULT_STOPWORDS)


def extract_keywords_simple(text: str, top_k: int = 8, min_len: int = 2) -> list:
    """간단한 정규식 기반 키워드 추출 (Mecab 불필요)"""
    if pd.isna(text) or not text:
        return []
    tokens = re.findall(r"[가-힣a-zA-Z0-9]+", text)
    filtered = [tok for tok in tokens if tok not in CUSTOM_STOPWORDS and len(tok) >= min_len]
    counts = Counter(filtered)
    return [token for token, _ in counts.most_common(top_k)]


# 키워드 추출
hot_df["keyword_list"] = (
    hot_df["title"].fillna("") + " " + hot_df["contents"].fillna("")
).apply(lambda x: extract_keywords_simple(x, top_k=8))
hot_df["keywords"] = hot_df["keyword_list"].apply(lambda ks: ", ".join(ks))

print("✅ 키워드 추출 완료")
print("샘플:")
display(hot_df[["title", "keywords"]].head(3))

---
## 6. 최종 학습 데이터셋 생성 (SOS/EOS + CoT 통합)

모든 RPM 결과물을 통합하여 SOS/EOS 토큰이 포함된
`training_dataset_v2.jsonl`을 생성합니다.

### 학습 데이터 포맷
```
<|begin_of_text|>Below is an instruction...

### Instruction:
주어진 키워드와 인기요인 정보를 사용하여 {board}의 인기글 스타일로 글을 작성해줘.

### Input:
키워드: ...
인기요인: ... (커버리지: ..., 영향도: ...)

### Thinking:
{reasoning_path}

### Response:
{title}\n\n{contents}<|end_of_text|>
```

In [ ]:
def build_training_record(
    row: pd.Series,
    reasoning_paths: dict,
    factors_data: dict,
) -> dict | None:
    """
    한 건의 학습 레코드를 생성합니다.
    RPM(피처/팩터/추론경로) + CoT + SOS/EOS 통합.
    """
    post_key = str(row.get("post_key", ""))
    keywords = row.get("keyword_list", [])
    if not keywords:
        return None

    board = str(row.get("board", "미상")).strip()
    title = str(row.get("title", "")).strip()
    contents = str(row.get("contents", "")).strip()
    full_text = f"{title}\n\n{contents}".strip()
    if not full_text or full_text == "nan":
        return None

    # ── 팩터 정보 ─────────────────────────────────────────
    rp_data = reasoning_paths.get(post_key, {})
    assigned_factors = rp_data.get("factors", [])
    reasoning = rp_data.get("reasoning", "")

    # 팩터 통계
    factor_stats = factors_data.get("factor_stats", {})
    if assigned_factors:
        primary_factor = assigned_factors[0]
        stats = factor_stats.get(primary_factor, {})
        coverage = stats.get("coverage", 0)
        factor_info = f"{primary_factor} (커버리지: {coverage:.0%})"
    else:
        factor_info = "일반"

    # ── Instruction / Input / Thinking / Output ────────────
    instruction = f"주어진 키워드와 인기요인 정보를 사용하여 {board}의 인기글 스타일로 글을 작성해줘."
    input_text = f"키워드: {', '.join(keywords)}\n인기요인: {factor_info}"

    # CoT (Thinking) - 추론경로
    thinking = reasoning if reasoning else f"{factor_info}에 해당하는 인기글을 작성합니다."

    return {
        "instruction": instruction,
        "input": input_text,
        "thinking": thinking,
        "output": full_text,
        "post_key": post_key,
        "board": board,
        "factors": assigned_factors,
    }


print("✅ 레코드 생성 함수 정의 완료")

In [ ]:
# ── 전체 데이터셋 생성 ────────────────────────────────────
print("🔧 학습 데이터셋 생성 중...\n")

records = []
skipped = 0

for _, row in tqdm(hot_df.iterrows(), total=len(hot_df), desc="데이터셋 생성"):
    record = build_training_record(row, reasoning_paths, factors_data)
    if record:
        records.append(record)
    else:
        skipped += 1

print(f"\n✅ 생성 완료: {len(records):,}건 (스킵: {skipped:,}건)")

# ── 게시판별 분포 ─────────────────────────────────────────
board_dist = Counter(r["board"] for r in records)
print("\n📊 게시판별 분포:")
for board, count in board_dist.most_common():
    print(f"  {board}: {count:,}건")

# ── 팩터별 분포 ───────────────────────────────────────────
factor_dist = Counter(
    f for r in records for f in r.get("factors", ["없음"])
)
print("\n📊 팩터별 분포:")
for factor, count in factor_dist.most_common():
    print(f"  {factor}: {count:,}건")

In [ ]:
# ── SOS/EOS 포함된 최종 JSONL 생성 ────────────────────────
# 참고: SOS/EOS 토큰은 Phase 3에서 토크나이저 로드 후
#       실제 토큰으로 대체됩니다. 여기서는 placeholder로 기록합니다.

BOS = "<|begin_of_text|>"
EOS = "<|end_of_text|>"

jsonl_path = OUTPUT_DIR / "training_dataset_v2.jsonl"

with open(jsonl_path, "w", encoding="utf-8") as f:
    for record in records:
        # 학습용 text 필드 (Phase 3에서 직접 사용)
        text = (
            f"{BOS}Below is an instruction that describes a task. "
            f"Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{record['instruction']}\n\n"
            f"### Input:\n{record['input']}\n\n"
            f"### Thinking:\n{record['thinking']}\n\n"
            f"### Response:\n{record['output']}{EOS}"
        )

        jsonl_record = {
            "instruction": record["instruction"],
            "input": record["input"],
            "thinking": record["thinking"],
            "output": record["output"],
            "text": text,
        }
        f.write(json.dumps(jsonl_record, ensure_ascii=False) + "\n")

print(f"✅ 학습 데이터셋 저장 완료: {len(records):,}건 → {jsonl_path}")

In [ ]:
# ── 샘플 확인 ─────────────────────────────────────────────
print("\n" + "=" * 70)
print("샘플 레코드 (최대 3건)".center(70))
print("=" * 70)

for i, rec in enumerate(records[:3], 1):
    print(f"\n[{i}] 게시판: {rec['board']} | 팩터: {', '.join(rec['factors'])}")
    print(f"  Instruction: {rec['instruction'][:80]}")
    print(f"  Input: {rec['input'][:80]}")
    print(f"  Thinking: {rec['thinking'][:120]}...")
    print(f"  Output: {rec['output'][:120]}...")
    print("-" * 70)

---
## 7. 완료 및 다음 단계

### 생성된 파일
| 파일 | 설명 |
|------|------|
| `rpm_features.json` | RPM Step 1: 각 게시글의 인기 기여 피처 |
| `rpm_factors.json` | RPM Step 2: 팩터(클러스터) + 할당 + 통계 |
| `rpm_reasoning_paths.json` | RPM Step 3: 추론경로 (= CoT) |
| `training_dataset_v2.jsonl` | 최종 학습 데이터셋 (SOS/EOS + CoT 포함) |

### 다음 단계
→ `phase3_finetune_v2.ipynb`에서 QLoRA 학습 진행

In [ ]:
print("\n" + "=" * 70)
print("Phase 2v2 완료 - RPM 기반 고도화 데이터셋 요약".center(70))
print("=" * 70)
print(f"  전체 인기글:   {len(hot_df):,}건")
print(f"  추출 피처:     {len(all_features):,}건")
print(f"  팩터 수:       {len(factor_names)}개")
print(f"  추론경로:      {len(reasoning_paths):,}건")
print(f"  학습 데이터셋: {len(records):,}건")
print("=" * 70)
print("\n🎉 Phase 2v2 완료!")
print("다음 단계: phase3_finetune_v2.ipynb")